<img style="float: left; padding-right: 20px; width: 70px" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg/1920px-Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg.png"> 

**Pontificia Universidad Católica de Chile**<br>
**Semestre 2021-2**<br>
**Profesora:** Paula Aguirre <br>

----
<div style="background-color:#fffff;">
    <br>
    <p style="text-align:center;font-family:Times New Roman;color:black;font-size:20pt;font-weight:bold;margin:20px">
        Introducción a la Ciencia de Datos: Proyecto Final
    </p>
    <p style="text-align:center;font-family:Times New Roman;color:black;font-size:20pt;font-weight:bold;margin:20px">
        Influencia de los factores sociales y económicos en la proyección de la
delincuencia.
    </p>
    <p style="text-align:center;font-family:Times New Roman;color:black;font-size:12pt;font-weight:bold;margin:20px">
        <br>Cristóbal Cancino - cristobal.cancinob@uc.cl</br>
        <br>Diego Rodríguez - drodrguez@uc.cl</br>
        <br>Benjamín Prieto - bprietz@uc.cl</br>
        <br>Larry Uribe - larry@uc.cl</br>
        <br>14 de diciembre de 2021
        <br><br><b>Tiempo de ejecución aproximado:</b> x minutos.
    </p>
    <br><br>
</div>

## El proyecto se divide en 7 etapas: 

1. Extracción de datos
        data_total_dmcs -> Delito
        data_educacion -> calidad educacion
        data_pobreza -> indicadores de pobreza
        data_comunas -> info de comunas
        data_natalidad -> info de natalidad (diccionario por años) 
        data_mortalidad -> info de mortalidad (diccionario por años)
        data_salud -> establecimientos MINSAL (mirar años)


2. Limpieza de datos

3. Procesamiento y transformación

4. Analisis exploratorio

5. Aprendizaje supervisado
        regresiones multilineal, ridge, lasso -> a predecir el total_dmcs

6. Aprendizaje no supervisado
        aplicaremos kmeans sobre mapa de chile o santiago (definir comunas) (puede ser uno  por pais y region, y otro por comunas de santiago)

7. Conclusiones y visualizaciones al cierre. 

## 1. Extraccion de datos:

In [1]:
import pandas as pd
import requests 
import os
import io, zipfile

In [2]:
#Cargar archivos (I) : Data sobre denuncias de crimen de mayor connotación social.

link_denuncias = 'http://cead.spd.gov.cl/centro-de-documentacion/?wpdmpro=estadisticas-anuales-delitos-de-mayor-connotacion-social-dmcs-base-censo-2017&wpdmdl=3047&'
rdenuncias = requests.get(link_denuncias, stream = True)

#Se necesita crear la carpeta para almacenar los archivos de datos.
#exist_ok permite evitar errores derivados de la ya-existencia de la carpeta.
ruta_delitos = os.path.join('Data')
os.makedirs(ruta_delitos, exist_ok = True)

#Descompresión del archivo necesario de todo el ZIP
z = zipfile.ZipFile(io.BytesIO(rdenuncias.content))
z.extractall(path=ruta_delitos, members=['Frecuencia Anual_DENUNCIA.xlsx'] )

#Crea dataframe
path_denuncias = os.path.join('Data', 'Frecuencia Anual_DENUNCIA.xlsx')
data_total_dmcs = pd.read_excel(path_denuncias, sheet_name = 'TOTAL_DMCS')
#columnas_denuncias = data_denuncias.keys()

# Carga archivos (II) : Data de calidad de educación.
link_educacion = 'http://datos.mineduc.cl/datasets/193132-bd-sned-2018-2019.download/'
data_educacion = pd.read_csv(link_educacion, delimiter = ';')

# Carga archivos (III): Data de indicador pobreza

link_pobreza = 'http://observatorio.ministeriodesarrollosocial.gob.cl/storage/docs/pobreza-comunal/2017/PLANILLA_Estimaciones_comunales_tasa_pobreza_por_ingresos_multidimensional_2017.xlsx'
data_pobreza = pd.read_excel(link_pobreza, sheet_name = 'Multidimensional 2017', engine='openpyxl')

# Cargar archivos (IV) : Información varía sobre comunas.
link_info_comunal = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/input/Otros/InformacionComunas.csv"
data_comunas = pd.read_csv(link_info_comunal)

# Cargar archivos (V) : Información sobre natalidad se genera dinamicamente lista de archivos
datas_natalidad = dict()

for ano in range(2011, 2021):
    link_natalidad_p = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/input/RegistroCivil/Nacimientos/Nacimientos_"+str(ano)+"-01-01_"+str(ano)+"-12-31_DO.csv"
    datas_natalidad[ano] = pd.read_csv(link_natalidad_p)
    
# Cargar archivos (VI): Información sobre la mortalidad. se carga dinamico por año

datas_mortalidad = dict()

for ano in range(2011, 2021):
    link_mortalidad_p = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/input/RegistroCivil/Defunciones/Defunciones_"+str(ano)+"-01-01_"+str(ano)+"-12-31_DO.csv"
    datas_mortalidad[ano] = pd.read_csv(link_mortalidad_p)

# Cargar archivos (VII) : Información sobre acceso a la salud
# Para medir el acceso a la salud solo se considerará la información actualizada al 2021, por el acceso a los datos;
# Y también por la poca variabilidad del nivel de hospitales

link_salud = 'https://repositoriodeis.minsal.cl/DatosAbiertos/Establecimientos_ChileDEIS_MINSAL%2029-10-2021.xlsx'
data_salud = pd.read_excel(link_salud, sheet_name = 'Establecimientos V')
#data_salud = data_salud[]    
  

## 2. Limpieza de datos:

In [3]:
# Conjunto de datos I:
# Limpia datos  de mayor connotacion social

data_total_dmcs = data_total_dmcs[1:]
data_total_dmcs.columns = data_total_dmcs.iloc[1]
data_total_dmcs = data_total_dmcs[data_total_dmcs['UN_ADMIN'] == 'COMUNA'] 
data_total_dmcs = data_total_dmcs[['UNIDAD', 2011.0, 2012.0, 2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2018.0, 2019.0, 2020.0]]
#revisar como eliminar el header del index
data_total_dmcs.reset_index(level=0, inplace=True, drop=True)
data_total_dmcs.rename(columns={'UNIDAD': 'Comuna', 2011.0: '2011', 2012.0: '2012', 2013.0: '2013', 2014.0: '2014', 2015.0: '2015', 2016.0: '2016', 2017.0: '2017', 2018.0: '2018', 2019.0: '2019', 2020.0: '2020'}, inplace=True)
data_total_dmcs


2,Comuna,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Arica,5750.0,5675.0,5524.0,5127.0,4735.0,4859.0,4303.0,4744.0,5244.0,3953.0
1,Camarones,29.0,34.0,28.0,24.0,19.0,21.0,15.0,24.0,27.0,23.0
2,General Lagos,8.0,3.0,5.0,8.0,10.0,10.0,8.0,7.0,7.0,5.0
3,Putre,30.0,30.0,28.0,34.0,34.0,25.0,27.0,21.0,30.0,14.0
4,Alto Hospicio,3597.0,3581.0,3288.0,2843.0,2924.0,2935.0,2858.0,2640.0,3005.0,2406.0
...,...,...,...,...,...,...,...,...,...,...,...
341,Porvenir,92.0,107.0,74.0,77.0,85.0,114.0,70.0,87.0,83.0,82.0
342,Primavera,15.0,13.0,9.0,16.0,18.0,8.0,6.0,16.0,7.0,13.0
343,Timaukel,5.0,6.0,1.0,1.0,1.0,1.0,2.0,4.0,2.0,4.0
344,Natales,380.0,413.0,352.0,361.0,363.0,354.0,285.0,258.0,249.0,162.0


In [4]:
data_educacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11025 entries, 0 to 11024
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   RBD             11025 non-null  int64 
 1   DV_RBD          11025 non-null  int64 
 2   NOM_RBD         11025 non-null  object
 3   RBD_A1          11025 non-null  object
 4   RBD_A2          11025 non-null  object
 5   COD_DEPE        11025 non-null  int64 
 6   COD_DEPE2       11025 non-null  int64 
 7   COD_REG_RBD     11025 non-null  int64 
 8   COD_PRO_RBD     11025 non-null  int64 
 9   COD_COM_RBD     11025 non-null  int64 
 10  NOM_COM_RBD     11025 non-null  object
 11  COD_DEPROV_RBD  11025 non-null  int64 
 12  NOM_DEPROV_RBD  11025 non-null  object
 13  RURAL_RBD       11025 non-null  int64 
 14  EFECTIVR        11025 non-null  object
 15  SUPERAR         11025 non-null  object
 16  INICIAR         11025 non-null  object
 17  MEJORAR         11025 non-null  object
 18  INTEGR

In [5]:
#data_educacion = data_educacion[data_total_dmcs['UN_ADMIN'] == 'COMUNA'] 
data_educacion = data_educacion[['NOM_COM_RBD', 'INDICER']]
data_educacion["INDICER"]=data_educacion["INDICER"].str.replace(',', '.').astype(float)

data_educacion.rename(columns={'NOM_COM_RBD': 'Comuna', 'INDICER': 'Calidad Educacion'}, inplace=True)
data_educacion.groupby(by=["Comuna"]).mean()

,Calidad Educacion
Comuna,
ALGARROBO,58.768497
ALHUÉ,64.371385
ALTO BIOBÍO,54.109610
ALTO DEL CARMEN,58.989017
ALTO HOSPICIO,67.169581
...,...
YUMBEL,63.766439
YUNGAY,66.587932
ZAPALLAR,71.240982


In [6]:
data_comunas

,Region,Codigo region,Comuna,Codigo comuna,Superficie_km2,Poblacion
0,Tarapacá,1,Iquique,1101,2289.53,223463.0
1,Tarapacá,1,Alto Hospicio,1107,571.19,129999.0
2,Tarapacá,1,Pozo Almonte,1401,13765.80,17395.0
3,Tarapacá,1,Camiña,1402,2199.83,1375.0
4,Tarapacá,1,Colchane,1403,4007.84,1583.0
...,...,...,...,...,...,...
341,Biobío,8,Hualpén,8112,53.77,97273.0
342,Biobío,8,San Pedro de la Paz,8108,112.86,145906.0
343,Biobío,8,Hualqui,8105,535.85,26201.0
344,Magallanes y de la Antártica Chilena,12,Río Verde,12103,9147.57,211.0


In [7]:
datas_natalidad[2014]

,Region,Codigo region,Comuna,Codigo comuna,Nacimientos,Fecha
0,Tarapacá,1,Iquique,1101,31,2014-01-02
1,Tarapacá,1,Iquique,1101,19,2014-01-03
2,Tarapacá,1,Iquique,1101,27,2014-01-06
3,Tarapacá,1,Iquique,1101,22,2014-01-07
4,Tarapacá,1,Iquique,1101,14,2014-01-08
...,...,...,...,...,...,...
21260,La Araucanía,9,Victoria,9211,4,2014-12-24
21261,La Araucanía,9,Victoria,9211,4,2014-12-26
21262,La Araucanía,9,Victoria,9211,9,2014-12-29
21263,La Araucanía,9,Victoria,9211,5,2014-12-30


In [8]:
header_row = 1 
data_salud.columns = data_salud.iloc[header_row]
data_salud.head()
# se utiliza group count

1,26-216,126216,No Aplica,No Aplica,12,Región De Magallanes y de la Antártica Chilena,12,SEREMI De Magallanes y la Antártica Chilena,No Perteneciente,Centro de Salud,...,NO,No Aplica,No Aplica,-54.93672,-67.61514,Privado,Vigente en operación,Baja Complejidad,Atención Abierta-Ambulatoria,NaN
0,Código Antiguo,Código Vigente,Código Madre,Código Nuevo Madre,Código Región,Nombre Región,Código Dependencia Jerárquica (SEREMI / Servic...,Dependencia Jerárquica (SEREMI / Servicio de S...,Pertenencia al SNSS,Tipo Establecimiento,...,Tiene Servicio de Urgencia,Tipo de Urgencia,Tipo de SAPU,LATITUD [Grados decimales],LONGITUD [Grados decimales],Tipo de Prestador Sistema de Salud,Estado de Funcionamiento,Nivel de Complejidad,Modalidad de Atención,NaN
1,26-216,126216,No Aplica,No Aplica,12,Región De Magallanes y de la Antártica Chilena,12,SEREMI De Magallanes y la Antártica Chilena,No Perteneciente,Centro de Salud,...,NO,No Aplica,No Aplica,-54.93672,-67.61514,Privado,Vigente en operación,Baja Complejidad,Atención Abierta-Ambulatoria,NaN
2,26-704,126704,No Aplica,No Aplica,12,Región De Magallanes y de la Antártica Chilena,26,Servicio de Salud Magallanes,Perteneciente,Hospital,...,SI,Urgencia Hospitalaria (UEH),No Aplica,-54.935209,-67.600393,Público,Vigente en operación,Baja Complejidad,Atención Cerrada-Hospitalaria,NaN
3,26-204,126204,No Aplica,No Aplica,12,Región De Magallanes y de la Antártica Chilena,26,Servicio de Salud Magallanes,Perteneciente,Hospital,...,SI,Urgencia Hospitalaria (UEH),No Aplica,-54.934374,-67.608895,Pendiente,Vigente en operación,Mediana Complejidad,Atención Cerrada-Hospitalaria,NaN
4,26-412,126412,No Aplica,No Aplica,12,Región De Magallanes y de la Antártica Chilena,26,Servicio de Salud Magallanes,Perteneciente,Posta de Salud Rural (PSR),...,NO,No Aplica,No Aplica,-53.640546,-69.645848,Público,Vigente en operación,Baja Complejidad,Atención Abierta-Ambulatoria,NaN


In [9]:
#ESTA PARTE ESTÁ INCOMPLETA. COMPLETAR EL 25-11.
#linkazo = 'http://datos.mineduc.cl/datasets/196387-directorio-de-establecimientos-2021.download/'
#headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36',
          #'Cookie':'_ga=GA1.2.377815330.1633997856; _hjid=bfe9d992-da76-49cd-a7a3-6c82b7d655c7; fpestid=xtL6ZQONtwqONtIca9x4o2WAPNfqzeiJaYoZp7J0OmqM_eJwGYVkPPc_OE1nj86MiKnffA; __utmz=212104321.1634046891.2.2.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not provided); _gid=GA1.2.1117445521.1637702821; __utma=212104321.377815330.1633997856.1634310102.1637702823.5; __utmc=212104321; _hjSessionUser_667338=eyJpZCI6IjI2MGZiYzk0LTI2ZjItNTI5My05NDNiLTBhMjFlYTM2YWU1YiIsImNyZWF0ZWQiOjE2Mzc3MDI4MjEzNzAsImV4aXN0aW5nIjp0cnVlfQ==; _hjSession_667338=eyJpZCI6IjlkZTdkNDJmLTUyNTEtNGI2NS1iNmE0LWU5YmQwYzAzODA4YSIsImNyZWF0ZWQiOjE2Mzc3MDI4MjMxNzR9; _hjAbsoluteSessionInProgress=0; __utmt=1; __utmb=212104321.2.10.1637702823; _hjIncludedInPageviewSample=1'}
#r = requests.get(linkazo)#, headers = headers)
#print (r.status_code)

#session = requests.Session()




